# GEOtop Calibration Report

In [ ]:
# Defaults
model_path = '../data/testbed/inputs'
timeout = 120
observations_path = '../data/testbed/observations/obs.csv'
parameters_path = '../data/parameters/testbed.csv'
default_parameters = {}
algorithm = 'Random'
popsize = 2
num_generations = 2
scheduler_file = None
num_cpus = 2
num_workers = 2
performance_report_filename='report.html'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scrapbook as sb
import dask.config
from dask.distributed import Client, performance_report

from mhpc_project.utils import date_parser, kge_cmp, calibrate, delta_mim
from mhpc_project.parameters import UniformSoilParameters as Parameters
from mhpc_project.models import UniformSoilModel as Model
import mhpc_project.plots as plots

In [ ]:
# Store dask config
sb.glue('dask_config', dask.config.config)

In [ ]:
parameters = Parameters(parameters_path, default_parameters)
model = Model(model_path, store=False, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)


In [ ]:
if scheduler_file:
    client = Client(scheduler_file=scheduler_file)
else:
    client = Client()
client.wait_for_workers(n_workers=num_workers, timeout=180)

In [ ]:
with performance_report(filename=performance_report_filename):
    recommendation, predictions, log = calibrate(model,
                                                 parameters,
                                                 observations,
                                                 algorithm,
                                                 popsize,
                                                 num_generations,
                                                 client,
                                                 num_cpus)

sb.glue('loss', kge_cmp(predictions, observations))
sb.glue('log', [((x.generation, x.args[0]), l, t) for (x, l, t) in log])

In [ ]:
plot = plots.convergence([(x.generation, l) for x, l, _ in log if np.isfinite(l)])
plt.close(plot)
sb.glue('convergence_plot', plot, 'display')

In [ ]:
for name, plot in plots.comparisons(predictions, observations).items():
    plt.close(plot)
    sb.glue(name + '_plot', plot, 'display')

In [ ]:
report = delta_mim(parameters, [(x.args[0], l) for x, l, t in log if np.isfinite(l)])
report['best'] = parameters.from_instrumentation(recommendation)
sb.glue('report', report, 'pandas')

report.sort_values('delta', key=np.abs, ascending=False)